In [ ]:
import cellxgene_census

In [20]:
#census_datasets.loc[tabula_liver,].head(1)

,soma_joinid,citation,collection_id,collection_name,collection_doi,collection_doi_label,dataset_id,dataset_version_id,dataset_title,dataset_h5ad_path,dataset_total_cell_count
6,444,Publication: https://doi.org/10.1038/s41586-02...,0b9d8a04-bb9d-44da-aa27-705bb65b54eb,Tabula Muris Senis,10.1038/s41586-020-2496-1,The Tabula Muris Consortium et al. (2020) Nature,4546e757-34d0-4d17-be06-538318925fcd,ab2c1368-ab62-44a2-8bb2-18e8999acd64,Liver - A single-cell transcriptomic atlas cha...,4546e757-34d0-4d17-be06-538318925fcd.h5ad,2859


In [7]:
# Default fallback to prevent huge queries
tissue = "decidua basalis"
cell_type_ontology_term_id = "CL:0000525"
dataset_id = "f171db61-e57e-4535-a06a-35d8b6ef8f2b"
is_primary_data = True
# Mapping of field names to values
filter_args = {
    "dataset_id": dataset_id,
    "cell_type_ontology_term_id": cell_type_ontology_term_id,
    "tissue": tissue
}

In [8]:
# Convert all values to list
filter_args = {k: v if isinstance(v, list) else [v] for k, v in filter_args.items() if v is not None}

# Build value_filter string
filters = []
if is_primary_data:
    filters.append("is_primary_data == True")
for k, v in filter_args.items():
    if v:
        quoted = [f'"{x}"' for x in v]
        filters.append(f"{k} in [{', '.join(quoted)}]")
value_filter = " and ".join(filters) if filters else None
value_filter

'is_primary_data == True and dataset_id in ["f171db61-e57e-4535-a06a-35d8b6ef8f2b"] and cell_type_ontology_term_id in ["CL:0000525"] and tissue in ["decidua basalis"]'

In [ ]:
census_datasets = (census["census_info"]["datasets"].read().concat().to_pandas())

In [34]:
census_datasets.to_csv("census_info_datasets.tsv", sep='\t', index=None)

In [17]:
census = cellxgene_census.open_soma(census_version="latest")
census_datasets = (
    census["census_info"]["datasets"].read(value_filter="collection_name == 'Tabula Muris Senis'").concat().to_pandas()
)
tabula_liver = census_datasets["dataset_title"].str.contains("liver", case=False)
census_datasets.loc[tabula_liver,].head(1)


,soma_joinid,citation,collection_id,collection_name,collection_doi,collection_doi_label,dataset_id,dataset_version_id,dataset_title,dataset_h5ad_path,dataset_total_cell_count
6,444,Publication: https://doi.org/10.1038/s41586-02...,0b9d8a04-bb9d-44da-aa27-705bb65b54eb,Tabula Muris Senis,10.1038/s41586-020-2496-1,The Tabula Muris Consortium et al. (2020) Nature,4546e757-34d0-4d17-be06-538318925fcd,ab2c1368-ab62-44a2-8bb2-18e8999acd64,Liver - A single-cell transcriptomic atlas cha...,4546e757-34d0-4d17-be06-538318925fcd.h5ad,2859


In [ ]:
species="homo_sapiens"
obs_df = cellxgene_census.get_obs(
            census,
            species,
            column_names=["dataset_id", "tissue_general"]
        )

obs_df = obs_df.drop_duplicates()
#.to_csv("census_info_id_per_tissue.tsv", sep='\t', index=None)



In [ ]:
# 3. Aggregate tissues by dataset_id
tissue_map = (
    obs_df.dropna(subset=["tissue_general"])
    .groupby("dataset_id", observed=False)["tissue_general"]
    .unique()
    .reset_index()
)
tissue_map["tissue_general"] = tissue_map["tissue_general"].apply(list)

# 4. Merge tissue info into datasets
merged = pd.merge(census_datasets, tissue_map, on="dataset_id", how="left")


/tmp/ipykernel_47729/2498269174.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("dataset_id")["tissue_general"]


NameError: name 'pd' is not defined

In [ ]:
    with cellxgene_census.open_soma(census_version=census_version) as census:

        obs_df = cellxgene_census.get_obs(
            census,
            species,
            value_filter=value_filter,
            column_names=column_names
        ).head(100)  # Limit to 100 rows to avoid timeout

    dataset_ids = sorted(obs_df["dataset_id"].dropna().unique().tolist())